In [ ]:
!pip install llama-index
!pip install llama-index-llms-groq
!pip install llama-index-embeddings-huggingface

  Using cached llama_index_embeddings_huggingface-0.6.0-py3-none-any.whl.metadata (458 bytes)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached llama_index_embedd

In [ ]:
!pip install google-search-results


  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32093 sha256=e4e173bda415343db709e2aa11be66f8157c6af536dba242471dfb0889c099f4
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
# Get Groq API key from Colab secrets
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')
serp_api_key = userdata.get('SERP_API_KEY')

In [ ]:
# from llama_index.llms.groq import Groq
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import Settings
# from llama_index.core.agent import ReActAgent
# from llama_index.core.tools import FunctionTool
# import nest_asyncio
# from google.colab import userdata
# from serpapi import GoogleSearch
# from typing import Dict, List
# import asyncio

# nest_asyncio.apply()

# # Configure Groq LLM
# Settings.llm = Groq(
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     api_key=userdata.get('GROQ_API_KEY'),
# )

# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="sentence-transformers/all-MiniLM-L6-v2"
# )
# Simplified Multi-Agent Research & Report System
from google.colab import userdata
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from serpapi import GoogleSearch
import asyncio
import nest_asyncio

nest_asyncio.apply()

# Configure LLM (use lighter model to reduce rate limits)
Settings.llm = Groq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Changed to lighter model
    api_key=userdata.get('GROQ_API_KEY'),
)

Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


In [ ]:
def web_search_real(query: str) -> str:
    """Real web search using SerpApi"""
    try:
        api_key = userdata.get('SERPAPI_API_KEY')

        params = {
            "q": query,
            "engine": "google",
            "api_key": serp_api_key,
            "num": 5,  # Get more results for better research
            "gl": "us",
            "hl": "en"
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        if "organic_results" in results:
            search_snippets = []
            for i, result in enumerate(results["organic_results"], 1):
                title = result.get("title", "")
                snippet = result.get("snippet", "")
                link = result.get("link", "")
                search_snippets.append(f"{i}. {title}\n{snippet}\nSource: {link}")

            return "\n\n".join(search_snippets)
        else:
            return f"No results found for: {query}"

    except Exception as e:
        return f"Search error: {str(e)}"

def gather_statistics(topic: str) -> str:
    """Gathers statistical data about a topic"""
    stats = {
        "AI": {"market_size": "$150B", "growth_rate": "37%", "adoption_rate": "67%"},
        "machine learning": {"projects": "85%", "success_rate": "73%", "roi": "15x"},
        "python": {"users": "10M+", "job_demand": "+22%", "salary": "$95k"},
        "blockchain": {"market_cap": "$1.2T", "adoption": "+45%", "projects": "3000+"},
        "cloud computing": {"market": "$480B", "growth": "+15%", "adoption": "94%"}
    }

    for key in stats:
        if key.lower() in topic.lower():
            formatted_stats = []
            for metric, value in stats[key].items():
                formatted_stats.append(f"{metric}: {value}")
            return f"Statistical data for {key}: " + ", ".join(formatted_stats)

    return f"General statistics for {topic}: High growth potential, increasing adoption, significant market opportunity"

# Create research tools
web_search_tool = FunctionTool.from_defaults(fn=web_search_real)
statistics_tool = FunctionTool.from_defaults(fn=gather_statistics)

# Research Agent
research_agent = ReActAgent(
    tools=[web_search_tool, statistics_tool],
    llm=Settings.llm,
    verbose=False,
    system_prompt=(
        "You are a Research Agent with access to real-time web search capabilities. "
        "Your role is to gather comprehensive, current information on any given topic. "
        "Use web search to find the latest data, trends, news, and developments. "
        "Also gather relevant statistics to support your research. "
        "Provide thorough research summaries with key findings and cite your sources. "
        "Focus on accuracy, recency, and credibility of information."
    )
)

In [ ]:
def data_analyzer(raw_data: str) -> str:
    """Analyzes data and extracts key insights"""
    # Simulate sophisticated data analysis
    insights = [
        "Primary trend: Significant growth trajectory observed",
        "Market dynamics: Strong adoption signals across sectors",
        "Key opportunities: Emerging market segments showing potential",
        "Risk assessment: Technology maturity and competitive landscape",
        "Performance indicators: Above-average growth metrics",
        "Future outlook: Positive momentum with scaling opportunities"
    ]

    return f"Comprehensive analysis reveals: {', '.join(insights)}. Data correlation shows strong positive indicators for market expansion and technological advancement."

def trend_predictor(current_data: str) -> str:
    """Predicts future trends based on current data"""
    predictions = [
        "Short-term (6-12 months): 25-40% growth expected",
        "Medium-term (1-2 years): Mainstream adoption likely",
        "Long-term (3-5 years): Market maturity and consolidation",
        "Technology evolution: Integration with existing ecosystems",
        "Cost optimization: 30-50% efficiency improvements expected",
        "Regulatory landscape: Clearer frameworks emerging"
    ]

    return f"Predictive analysis indicates: {', '.join(predictions)}. Confidence level: High, based on current market indicators and adoption patterns."

# Create analysis tools
analyzer_tool = FunctionTool.from_defaults(fn=data_analyzer)
predictor_tool = FunctionTool.from_defaults(fn=trend_predictor)

# Analysis Agent
analysis_agent = ReActAgent(
    tools=[analyzer_tool, predictor_tool],
    llm=Settings.llm,
    verbose=False,
    system_prompt=(
        "You are an Analysis Agent. Your task is simple and focused:\n"
        "1. Use the data_analyzer tool to extract key insights from research data\n"
        "2. Use the trend_predictor tool to make future predictions\n"
        "3. Combine the results into a clear analytical summary\n"
        "4. STOP when you have used both tools and created your summary\n\n"
        "Be concise and direct. Do not overthink the analysis process."
    )
)

In [ ]:
def format_report(content: str, style: str = "professional") -> str:
    """Formats content into a structured report"""
    if style == "executive":
        return f"""
EXECUTIVE SUMMARY
{'='*60}
{content}

KEY RECOMMENDATIONS:
• Strategic investment recommended based on analysis
• Monitor market developments closely
• Consider partnership opportunities
• Prepare for scaling requirements

NEXT STEPS:
• Detailed implementation planning
• Stakeholder alignment
• Resource allocation
• Timeline development
"""
    else:
        return f"""
COMPREHENSIVE REPORT
{'='*60}
{content}

DETAILED FINDINGS:
Analysis shows strong market indicators and growth potential.
Recommendations include strategic positioning and resource optimization.

CONCLUSION:
Based on comprehensive research and analysis, opportunities
for growth and development are significant with proper execution.
"""

def create_visualizations(data: str) -> str:
    """Creates text-based visualizations and charts"""
    return """
TREND VISUALIZATION:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Year 2022: ████████░░░░░░░░░░ (40% baseline)
Year 2023: ██████████████░░░░ (70% growth)
Year 2024: ████████████████████ (100% projected)
Year 2025: ████████████████████████ (120% forecast)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

KEY METRICS DASHBOARD:
┌─────────────────────────────────────────────┐
│ Growth Rate:      ▲ 37%                     │
│ Market Size:      ▲ $150B                   │
│ Adoption Rate:    ▲ 67%                     │
│ Success Rate:     ▲ 73%                     │
│ ROI Potential:    ▲ 15x                     │
└─────────────────────────────────────────────┘
"""

# Create writing tools
formatter_tool = FunctionTool.from_defaults(fn=format_report)
visualization_tool = FunctionTool.from_defaults(fn=create_visualizations)

# Writer Agent
writer_agent = ReActAgent(
    tools=[formatter_tool, visualization_tool],
    llm=Settings.llm,
    verbose=False,
    system_prompt=(
        "You are a Writer Agent. Your task is straightforward:\n"
        "1. Use format_report tool to structure the content professionally\n"
        "2. Use create_visualizations tool if charts would be helpful\n"
        "3. Combine the formatted content into a final report\n"
        "4. Present the final report and STOP\n\n"
        "Keep it simple and direct. Focus on clear, professional formatting."
    )
)

In [ ]:
class MultiAgentOrchestrator:
    def __init__(self, research_agent, analysis_agent, writer_agent):
        self.research_agent = research_agent
        self.analysis_agent = analysis_agent
        self.writer_agent = writer_agent

    async def generate_report(self, topic: str, report_type: str = "comprehensive"):
        """Orchestrates the multi-agent workflow"""

        print(f"🚀 MULTI-AGENT REPORT GENERATION STARTED")
        print(f"📋 Topic: {topic}")
        print(f"📊 Report Type: {report_type}")
        print("="*70)

        # Phase 1: Research
        print(f"🔍 PHASE 1: Research Agent gathering real-time information...")
        research_result = await self.research_agent.run(
            f"Conduct comprehensive research on '{topic}'. "
            f"Use web search to find the latest developments, trends, news, and key information. "
            f"Also gather relevant statistics and market data. "
            f"Focus on current and credible sources."
        )
        print(f"✅ Research completed: {len(str(research_result))} characters of data collected")
        print()

        # Phase 2: Analysis
        print(f"📊 PHASE 2: Analysis Agent processing research data...")
        analysis_result = await self.analysis_agent.run(
            f"Analyze this comprehensive research data about '{topic}': "
            f"\n\nRESEARCH DATA:\n{research_result}\n\n"
            f"Extract key insights, identify patterns and trends, assess opportunities and risks, "
            f"and make future predictions. Provide detailed analytical conclusions.",
            max_iterations=40  # Increased from default 20
        )
        print(f"✅ Analysis completed: Strategic insights extracted")
        print()

        # Phase 3: Report Writing
        print(f"✍️ PHASE 3: Writer Agent creating final report...")
        final_report = await self.writer_agent.run(
            f"Create a {report_type} report on '{topic}' using the following information:\n\n"
            f"RESEARCH FINDINGS:\n{research_result}\n\n"
            f"ANALYSIS RESULTS:\n{analysis_result}\n\n"
            f"Format this into a professional report with clear sections, "
            f"include visualizations if relevant, and ensure actionable insights are highlighted. "
            f"Make it executive-ready and well-structured.",
            max_iterations=40  # Increased from default 20
        )
        print(f"✅ Report generation completed")
        print()

        return {
            "topic": topic,
            "report_type": report_type,
            "research_data": str(research_result),
            "analysis": str(analysis_result),
            "final_report": str(final_report),
            "word_count": len(str(final_report).split()),
            "summary": f"Multi-agent report on {topic} generated successfully"
        }

# Create the orchestrator
orchestrator = MultiAgentOrchestrator(research_agent, analysis_agent, writer_agent)

In [ ]:
async def run_multi_agent_report(topic: str, report_type: str = "comprehensive"):
    """Main function to run the multi-agent report generation"""

    try:
        # Generate the report
        result = await orchestrator.generate_report(topic, report_type)

        # Display results
        print("="*70)
        print("🎉 MULTI-AGENT REPORT GENERATION COMPLETED")
        print("="*70)
        print(f"Topic: {result['topic']}")
        print(f"Word Count: {result['word_count']} words")
        print("="*70)
        print("📄 FINAL REPORT:")
        print("="*70)
        print(result["final_report"])
        print("="*70)

        return result

    except Exception as e:
        print(f"❌ Error in multi-agent system: {str(e)}")
        return None

In [ ]:
async def demo_multi_agent_system():
    """Demo function showing different report types"""

    topics = [
        "Artificial Intelligence trends in 2024",
        "Sustainable energy technologies",
        "Future of remote work"
    ]

    print("🚀 MULTI-AGENT SYSTEM DEMONSTRATION")
    print("="*70)

    for i, topic in enumerate(topics, 1):
        print(f"\n🔢 DEMO {i}/{len(topics)}: {topic}")
        report_type = "executive" if i == 1 else "comprehensive"

        result = await run_multi_agent_report(topic, report_type)

        if result:
            print(f"✅ Demo {i} completed successfully")
        else:
            print(f"❌ Demo {i} failed")

        print("\n" + "="*70)

In [ ]:
# Example 1: Single Report Generation
async def main():
    """Main execution function"""

    # Generate a report on AI trends
    await run_multi_agent_report(
        topic="Latest developments in Artificial Intelligence 2024",
        report_type="executive"
    )

# Example 2: Multiple Reports Demo
# Uncomment the line below to run multiple demos
# asyncio.run(demo_multi_agent_system())

# Run the main example
asyncio.run(main())

🚀 MULTI-AGENT REPORT GENERATION STARTED
📋 Topic: Latest developments in Artificial Intelligence 2024
📊 Report Type: executive
🔍 PHASE 1: Research Agent gathering real-time information...
✅ Research completed: 1445 characters of data collected

📊 PHASE 2: Analysis Agent processing research data...


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-529' coro=<Workflow.run.<locals>._run_workflow() done, defined at /usr/local/lib/python3.11/dist-packages/workflows/workflow.py:365> exception=InvalidStateError('invalid state')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/workflows/context/context.py", line 689, in _step_worker
    new_ev = await instrumented_step(**kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/llama_index_instrumentation/dispatcher.py", line 368, in async_wrapper
    result = await func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/llama_index/core/agent/workflow/base_agent.py", line 407, in parse_agent_output
    raise WorkflowRuntimeError(
workflows.errors.WorkflowRuntimeError: Max iterations of 40 reached! Either something went wrong, or you can increase the max iterations with `

✅ Analysis completed: Strategic insights extracted

✍️ PHASE 3: Writer Agent creating final report...
❌ Error in multi-agent system: Error in step 'run_agent_step': Error code: 429 - {'error': {'message': 'Rate limit reached for model `meta-llama/llama-4-scout-17b-16e-instruct` in organization `org_01jztkwzvcfm1rb7dfngj6wbkv` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 497699, Requested 9833. Please try again in 21m41.4664s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


In [ ]:
from google.colab import userdata
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from serpapi import GoogleSearch
import asyncio
import nest_asyncio

nest_asyncio.apply()

# Configure LLM (use lighter model to reduce rate limits)
Settings.llm = Groq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Changed to lighter model
    api_key=userdata.get('GROQ_API_KEY'),
)

Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def quick_search(query: str) -> str:
    """Focused web search - returns top 3 results only"""
    try:
        search = GoogleSearch({
            "q": query,
            "api_key": userdata.get('SERP_API_KEY'),
            "num": 3,  # Reduced from 5
            "engine": "google"
        })
        results = search.get_dict()

        if "organic_results" in results:
            snippets = []
            for result in results["organic_results"]:
                title = result.get("title", "")
                snippet = result.get("snippet", "")[:200]  # Limit snippet length
                snippets.append(f"{title}: {snippet}")
            return "\n".join(snippets)
        return f"No results for: {query}"
    except Exception as e:
        return f"Search failed: {str(e)}"

def simple_analysis(data: str) -> str:
    """Simple data analysis with key insights"""
    return f"Key insights from data: Growth trend positive, market adoption increasing, opportunities in emerging sectors. Technology shows strong potential."

def format_basic_report(content: str) -> str:
    """Basic report formatting"""
    return f"""
RESEARCH REPORT
===============
{content}

SUMMARY
=======

"""

In [ ]:
# === SIMPLIFIED AGENTS ===

# Research Agent - Shorter prompt
research_agent = ReActAgent(
    tools=[FunctionTool.from_defaults(fn=quick_search)],
    llm=Settings.llm,
    verbose=False,
    # system_prompt="Research Agent: Use quick_search to find current info on the topic. Keep it focused and brief."
    system_prompt=(
        "Research Agent: Use quick_search ONCE to find info on the topic. "
        "After getting search results, immediately provide a summary. "
        "Do NOT search multiple times. STOP after one search."
    )
)

# Analysis Agent - Shorter prompt
analysis_agent = ReActAgent(
    tools=[FunctionTool.from_defaults(fn=simple_analysis)],
    llm=Settings.llm,
    verbose=False,
    system_prompt="Analysis Agent: Use simple_analysis to extract key insights. Be concise."

)

# Writer Agent - Shorter prompt
writer_agent = ReActAgent(
    tools=[FunctionTool.from_defaults(fn=format_basic_report)],
    llm=Settings.llm,
    verbose=False,
    # system_prompt="Writer Agent: Use format_basic_report to create final report. Keep it simple."
    system_prompt="Writer Agent: Use format_basic_report ONCE to create final report. After using the tool, present the result and STOP. Do not overthink."
)

In [ ]:
# === SIMPLIFIED ORCHESTRATOR ===

class SimpleOrchestrator:
    def __init__(self, research_agent, analysis_agent, writer_agent):
        self.research_agent = research_agent
        self.analysis_agent = analysis_agent
        self.writer_agent = writer_agent

    async def generate_report(self, topic: str):
        """Streamlined 3-step workflow"""

        print(f"🔍 Step 1: Researching '{topic}'...")
        # Shorter, focused research prompt
        research = await self.research_agent.run(f"Search for latest info on: {topic}")

        print(f"📊 Step 2: Analyzing data...")
        # Simplified analysis prompt
        analysis = await self.analysis_agent.run(f"Analyze: {str(research)[:500]}")  # Limit input length

        print(f"✍️ Step 3: Writing report...")
        # Concise writing prompt
        report = await self.writer_agent.run(f"Create report on {topic} using: {str(analysis)[:300]}")

        return {
            "topic": topic,
            "final_report": str(report),
            "word_count": len(str(report).split())
        }

In [ ]:
 #=== EXECUTION ===

orchestrator = SimpleOrchestrator(research_agent, analysis_agent, writer_agent)

async def run_simple_demo(topic: str):
    """Simple demo function"""
    try:
        print(f"🚀 Multi-Agent Demo: {topic}")
        print("="*50)

        result = await orchestrator.generate_report(topic)

        print("="*50)
        print("📄 FINAL REPORT:")
        print("="*50)
        print(result["final_report"])
        print(f"\n✅ Complete! ({result['word_count']} words)")

        return result

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

In [ ]:
# === MAIN EXECUTION ===

async def main():
    # Single focused demo
    await run_simple_demo("AI trends 2024")

# Run it
asyncio.run(main())

🚀 Multi-Agent Demo: AI trends 2024
🔍 Step 1: Researching 'AI trends 2024'...
📊 Step 2: Analyzing data...
✍️ Step 3: Writing report...
📄 FINAL REPORT:
RESEARCH REPORT
AI Trends for2024: A Year of Growth and Innovation

The current AI trends for2024 indicate a positive growth trend, with increasing market adoption and opportunities in emerging sectors. The technology shows strong potential, with key areas of focus including:

1. Diversification of AI benchmarks to move beyond saturation.
2. Advancements in transformer-based models and their applications.

These trends are expected to drive innovation and adoption across industries, leading to new applications and use cases for AI.

SUMMARY
Based on analysis, positive outlook with growth opportunities. 

The AI landscape for 2024 is expected to be shaped by these trends, leading to increased adoption and innovation in the field. Key areas to watch include the development of more diverse and comprehensive benchmarks, as well as the continu

In [ ]:
# === MAIN EXECUTION ===

async def main():
    # Single focused demo
    await run_simple_demo("Best thriller/mystery novels.")

# Run it
asyncio.run(main())

🚀 Multi-Agent Demo: Best thriller/mystery novels.
🔍 Step 1: Researching 'Best thriller/mystery novels.'...
📊 Step 2: Analyzing data...
✍️ Step 3: Writing report...
📄 FINAL REPORT:
## Best Thriller/Mystery Novels of2023

The list of thriller/mystery novels from2023 includes titles with dark and ominous themes, and some focus on family or relationships. The authors are all well-known in the genre.

### Notable Novels:
Some of the notable novels in this genre include:
- Verity by Colleen Hoover: A dark and twisted tale of family secrets and lies.
- The Last Mrs. Parrish by Liv Constantine: A mysterious story of relationships and deceit.
- The Silent Patient by Alex Michaelides: A thrilling ride of suspense and betrayal.

### Common Themes:
The common themes found in these novels are:
- Dark and ominous themes
- Family secrets and lies
- Relationships and deceit
- Suspense and betrayal

### Well-Known Authors:
Some of the well-known authors in this genre are:
- Colleen Hoover: Known for he

In [ ]:
# === MAIN EXECUTION ===

async def main():
    # Single focused demo
    await run_simple_demo("Best songs of 2022")

# Run it
asyncio.run(main())

🚀 Multi-Agent Demo: Best songs of 2022
🔍 Step 1: Researching 'Best songs of 2022'...
❌ Error: Error in step 'parse_agent_output': Max iterations of 20 reached! Either something went wrong, or you can increase the max iterations with `.run(.., max_iterations=...)`


In [ ]:
# === MAIN EXECUTION ===

async def main():
    # Single focused demo
    await run_simple_demo("2022 Billboard charts official")

# Run it
asyncio.run(main())

🚀 Multi-Agent Demo: 2022 Billboard charts official
🔍 Step 1: Researching '2022 Billboard charts official'...
📊 Step 2: Analyzing data...
✍️ Step 3: Writing report...
📄 FINAL REPORT:
RESEARCH REPORT
##2022 Billboard Charts Report

### Overview of2022 Billboard Charts
The year2022 saw significant trends in music chart performance, with various artists and songs making notable impacts. Among these, 'Heat Waves' by Glass Animals emerged as a standout hit.

### Performance of 'Heat Waves' by Glass Animals
- **Chart-Topping Success**: 'Heat Waves' was the best-performing single of2022.
- **Weeks at Top**: The song spent5 weeks at the top of the Billboard Hot100.
- **Total Weeks on Chart**: It accumulated a total of91 weeks on the Billboard Hot100 chart.

### Analysis
This performance indicates a significant and sustained popularity of 'Heat Waves' across various digital music platforms and radio airplay. The song's ability to stay at the top for5 weeks and remain on the chart for91 weeks dem

In [ ]:
# === MAIN EXECUTION ===

async def main():
    # Single focused demo
    await run_simple_demo("A small report on real estate market analysis")

# Run it
asyncio.run(main())

🚀 Multi-Agent Demo: A small report on real estate market analysis
🔍 Step 1: Researching 'A small report on real estate market analysis'...
📊 Step 2: Analyzing data...
✍️ Step 3: Writing report...
📄 FINAL REPORT:
The current real estate market analysis indicates a mixed trend. On one hand, there is a 4.1% year-over-year increase in homes sold, and a significant 12.5% increase in single-family home sales. This suggests a positive outlook with growth opportunities, particularly in the single-family home segment.

However, housing market predictions indicate that while there will be more sales activity, prices may remain flat. This could be due to various factors such as changes in supply and demand, economic conditions, and government policies. As a result, investors and buyers are advised to adopt a cautious approach.

The flat prices could be a result of a balance between the number of homes being sold and the number of buyers in the market. It could also indicate a saturation point, wh